In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

Copying the code cells from the previous exercise's notebook. Every cell from this cell to the next markdwon cell is straight from the exercise_7.ipynb notebook.

In [23]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10_000, noise=0.4, random_state=42)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_moons, y_moons, test_size=0.2, random_state=42
)

a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: You can use Scikit-learn `ShuffleSplit` class for this.

In [25]:
from sklearn.model_selection import ShuffleSplit

n_instances = 100
n_tress = 1000

mini_sets = []

random_splits = ShuffleSplit(n_splits=n_tress, train_size=n_instances, random_state=42)

for mini_train_index, mini_test_index in random_splits.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy

A note that the best hyperparamter set found in the previous notebook is `DecisionTreeClassifier(max_depth=6, max_leaf_nodes=17, min_samples_split=2, random_state=42)`.

In [26]:
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

forest: list[DecisionTreeClassifier] = [
    clone(
        DecisionTreeClassifier(
            max_depth=6, max_leaf_nodes=17, min_samples_split=2, random_state=42
        )
    )
    for _ in range(n_tress)
]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_predict = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_predict))

np.mean(accuracy_scores)

0.8056605

c. Now come the magic. For each test set generate, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy's `mode` function for this). This approach gives you *majority-vote predictions* over the tets set.

In [27]:
Y_predict = np.empty([n_tress, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_predict[tree_index] = tree.predict(X_test)

In [28]:
from scipy.stats import mode

y_predict_major_votes, n_votes = mode(Y_predict, axis=0)

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5% to 1.5% higher). Congratulations, you have trained a random forest classifier!

In [29]:
accuracy_score(y_test, y_predict_major_votes.reshape([-1]))

0.873